In [1]:
import sympy as sp
from pprint import pprint
import numpy as np

# FIRST question 

In [2]:
def taylor_series(f, point, epsilon):
    # Define the symbolic variable
    x = sp.symbols('x')
    
    # Initialize variables
    s = 0.0  # Numerical sum of the Taylor series
    i = 0  # Current order of derivative
    term = epsilon + 1  # Ensure at least one iteration
    
    while abs(term) > epsilon:
        # Compute the i-th derivative symbolically
        f_prime = sp.diff(f, x, i)
        
        # Evaluate the i-th term numerically
        term = float(f_prime.subs(x, point)) / sp.factorial(i)
        
        # Add the term to the series
        s += term
        
        # Move to the next order
        i += 1
    
    return s

# Example usage
# part 1,2
epsilon = 1e-7
x = sp.symbols('x')
f = sp.sin(x)
result = taylor_series(f, 1, epsilon)  # Taylor series for sin(x) at x=1 with epsilon
prec = int(abs(np.log10(epsilon)))
print(f'result is {result:.{prec}}')

result is 0.9092974


# second question

###### part 1

In [3]:
# Define the function f(x, y1, y2, y3)
def f(x, y1, y2, y3):
    return (-y3 / 4) + (y2 / 16) - (y1 / 32) + \
           3 * np.sin(x / 2) + 2 * np.cos(x / 2) + 0.1875 * np.exp(x)

# Runge-Kutta method for a system of first-order ODEs
def runge_kutta_3rd_order(f, y0, x0, x_end, h):
    # Number of steps
    n = int((x_end - x0) / h)
    
    # Initialize arrays to store results
    x_values = np.zeros(n + 1)
    y_values = np.zeros((n + 1, 3))  # 3 for y1, y2, y3

    # Set initial conditions
    x_values[0] = x0
    y_values[0] = y0

    # Runge-Kutta iteration
    for i in range(n):
        x = x_values[i]
        y1, y2, y3 = y_values[i]

        # Calculate k values
        k1 = h * y2
        l1 = h * y3
        m1 = h * f(x, y1, y2, y3)

        k2 = h * (y2 + 0.5 * l1)
        l2 = h * (y3 + 0.5 * m1)
        m2 = h * f(x + 0.5 * h, y1 + 0.5 * k1, y2 + 0.5 * l1, y3 + 0.5 * m1)

        k3 = h * (y2 + 0.5 * l2)
        l3 = h * (y3 + 0.5 * m2)
        m3 = h * f(x + 0.5 * h, y1 + 0.5 * k2, y2 + 0.5 * l2, y3 + 0.5 * m2)

        k4 = h * (y2 + l3)
        l4 = h * (y3 + m3)
        m4 = h * f(x + h, y1 + k3, y2 + l3, y3 + m3)

        # Update values
        y_values[i + 1, 0] = y1 + k1 / 6 + k2 / 3 + k3 / 3 + k4 / 6
        y_values[i + 1, 1] = y2 + l1 / 6 + l2 / 3 + l3 / 3 + l4 / 6
        y_values[i + 1, 2] = y3 + m1 / 6 + m2 / 3 + m3 / 3 + m4 / 6

        x_values[i + 1] = x + h

    return x_values, y_values

# Initial conditions
x0 = -15  # Initial x
x_end = 10.0  # End x
h = 5  # Step size
y0 = np.array([12, -3, -5.75])  # Initial conditions: y(0), y'(0), y''(0)

# Run the Runge-Kutta method
x_values, y_values = runge_kutta_3rd_order(f, y0, x0, x_end, h)
print(np.round(y_values))
#x_values     array([-15., -10.,  -5.,   0.,   5.,  10.])

# y_values     [[ 1.200e+01 -3.000e+00 -6.000e+00]
#  [-5.600e+01 -2.000e+01  6.000e+00]
#  [-5.000e+01  2.700e+01  6.000e+00]
#  [ 1.330e+02  3.400e+01  2.000e+00]
#  [ 3.350e+02  4.600e+01  2.100e+01]
#  [ 4.244e+03  2.054e+03  4.249e+03]]



[[ 1.200e+01 -3.000e+00 -6.000e+00]
 [-5.600e+01 -2.000e+01  6.000e+00]
 [-5.000e+01  2.700e+01  6.000e+00]
 [ 1.330e+02  3.400e+01  2.000e+00]
 [ 3.350e+02  4.600e+01  2.100e+01]
 [ 4.244e+03  2.054e+03  4.249e+03]]


###### part 2

In [4]:
def forward_difference_table(x, y):
    n = len(y)
    # Create a table for forward differences
    diff_table = np.zeros((n, n))
    diff_table[:, 0] = y  # First column is y values

    # Calculate forward differences
    for j in range(1, n):
        for i in range(n - j):
            diff_table[i][j] = diff_table[i + 1][j - 1] - diff_table[i][j - 1]

    return diff_table

def newton_forward_interpolation_polynomial(x, y):
    n = len(x)
    diff_table = forward_difference_table(x, y)

    # Define the symbol for x
    x_symbol = sp.symbols('x')

    # Initialize the polynomial
    polynomial = 0

    # Construct the polynomial using forward differences
    for i in range(n):
        term = diff_table[0][i]  # First element of the ith column
        # Construct the product term
        product_term = 1
        for j in range(i):
            product_term *= (x_symbol - x[j])  # (x - x_j)
        term *= product_term / sp.factorial(i)  # Divide by i!
        polynomial += term  # Add to the polynomial

    return polynomial

# Example usage
x_values = x_values.flatten()  # Known x values
y_values = y_values.flatten()# Corresponding y values

# Get the polynomial
polynomial = newton_forward_interpolation_polynomial(x_values, y_values)

# Display the polynomial
print(f"Newton Forward Interpolation Polynomial: {polynomial}")
"""

Newton Forward Interpolation Polynomial: -3.50635525823873*x*(x - 5.0)*(x + 5.0)*(x + 10.0)*(x + 15.0) + 8.01663201866367*x*(x + 5.0)*(x + 10.0)*(x + 15.0) - 15.0*x - 9.90585725546426*(x + 5.0)*(x + 10.0)*(x + 15.0) + 6.125*(x + 10.0)*(x + 15.0) - 213.0


"""

Newton Forward Interpolation Polynomial: -3.50635525823873*x*(x - 5.0)*(x + 5.0)*(x + 10.0)*(x + 15.0) + 8.01663201866367*x*(x + 5.0)*(x + 10.0)*(x + 15.0) - 15.0*x - 9.90585725546426*(x + 5.0)*(x + 10.0)*(x + 15.0) + 6.125*(x + 10.0)*(x + 15.0) - 213.0


'\n\nNewton Forward Interpolation Polynomial: -3.50635525823873*x*(x - 5.0)*(x + 5.0)*(x + 10.0)*(x + 15.0) + 8.01663201866367*x*(x + 5.0)*(x + 10.0)*(x + 15.0) - 15.0*x - 9.90585725546426*(x + 5.0)*(x + 10.0)*(x + 15.0) + 6.125*(x + 10.0)*(x + 15.0) - 213.0\n\n\n'

###### part 3

In [5]:
x = sp.symbols('x')
y_prime = sp.diff(polynomial,x)

def newton_raphson(f, df, x0, tol=1e-7, max_iter=100):
    x_n = x0
    for i in range(max_iter):
        # Evaluate the function and its derivative at the current guess
        f_xn = f.subs(sp.symbols('x'), x_n)
        df_xn = df.subs(sp.symbols('x'), x_n)
        
        # Check if the derivative is zero to avoid division by zero
        if df_xn == 0:
            print("Derivative is zero. No solution found.")
            return None
        
        # Update the guess using the Newton-Raphson formula
        x_n1 = x_n - f_xn / df_xn
        
        # Check for convergence
        if abs(x_n1 - x_n) < tol:
            return x_n1
        
        x_n = x_n1
    
    print("Maximum iterations reached. No solution found.")
    return None



# Initial guess
initial_guess = 2.0

# Find the root
root = newton_raphson(polynomial, y_prime, initial_guess)

if root is not None:
    print(f"Root found: {root}")
    
"""
Root found: 0.372092203280713
"""

Root found: 0.372092203280713


'\nRoot found: 0.372092203280713\n'

# third question

In [7]:
def newton_solver(f, g, h, epsilon, init=[1, 1, 1]):
    # Ensure the initial guess is a float array
    init = np.array(init, dtype=float)
    x, y, z = sp.symbols('x y z')  # Define symbols for SymPy

    # Convert input functions to symbolic expressions
    f_expr = f(x, y, z)
    g_expr = g(x, y, z)
    h_expr = h(x, y, z)

    # Compute symbolic gradients (Jacobian entries)
    grad_f = [sp.diff(f_expr, var) for var in (x, y, z)]
    grad_g = [sp.diff(g_expr, var) for var in (x, y, z)]
    grad_h = [sp.diff(h_expr, var) for var in (x, y, z)]

    # Lambdify functions for numerical evaluation
    f_num = sp.lambdify((x, y, z), f_expr, 'numpy')
    g_num = sp.lambdify((x, y, z), g_expr, 'numpy')
    h_num = sp.lambdify((x, y, z), h_expr, 'numpy')

    grad_f_num = [sp.lambdify((x, y, z), grad, 'numpy') for grad in grad_f]
    grad_g_num = [sp.lambdify((x, y, z), grad, 'numpy') for grad in grad_g]
    grad_h_num = [sp.lambdify((x, y, z), grad, 'numpy') for grad in grad_h]

    while True:
        # Evaluate gradients at the current point
        grad_f_val = [grad(*init) for grad in grad_f_num]
        grad_g_val = [grad(*init) for grad in grad_g_num]
        grad_h_val = [grad(*init) for grad in grad_h_num]

        # Create Jacobian matrix
        A = np.array([grad_f_val, grad_g_val, grad_h_val])

        # Evaluate function values at the current point
        b = np.array([-f_num(*init), -g_num(*init), -h_num(*init)])

        # Solve the linear system A * delta = b
        delta = np.linalg.solve(A, b)

        # Update the solution
        init = init + delta

        # Check for convergence
        if np.linalg.norm(delta) < epsilon:
            return init

# Example usage
solution = newton_solver(
    f:=lambda x, y, z: sp.symbols('x')**2 + sp.symbols('y')**2 + sp.symbols('z')**2 - 1,  # First equation
    g:=lambda x, y, z: sp.sin(sp.symbols('y')) * sp.symbols('z'),       # Second equation
    h:=lambda x, y, z: sp.symbols('x')**3 +  - sp.symbols('z'),       # Third equation
    epsilon=1e-5,
    init=[0.1, 0.1, 0.1]
)

print("Solution:", solution,"\n")
x, y, z = solution
###  Solution: [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149]
print(f(x,y,z),solution,"\n")
print(g(x,y,z),solution,"\n")
print(h(x,y,z),solution,"\n")
### they are so close 
'''


Solution: [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 

x**2 + y**2 + z**2 - 1 [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 

z*sin(y) [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 

x**3 - z [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 





'''

Solution: [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 

x**2 + y**2 + z**2 - 1 [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 

z*sin(y) [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 

x**3 - z [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] 



'\n\n\nSolution: [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] \n\nx**2 + y**2 + z**2 - 1 [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] \n\nz*sin(y) [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] \n\nx**3 - z [ 1.71602449e-005 -1.00000000e+000  2.25305659e-149] \n\n\n\n\n\n'

# forth question 

In [8]:
def krylov_method(A, v):
    n = A.shape[0]  # Dimension of the square matrix A
    # Initialize the Krylov matrix
    K = v  # First column of the Krylov matrix is v
    for i in range(1, n):
        K = np.c_[K, np.linalg.matrix_power(A, i).dot(v)]  # Append columns
    # Solve the linear system K * c = -A^n * v
    b = -np.linalg.matrix_power(A, n).dot(v)
    c = np.linalg.solve(K, b)
    return c

A = np.array([[1, 2, 3, 4],
              [2, 1, 2, 3],
              [3, 2, 1, 2],
              [4, 3, 2, 1]])
v = np.array([[1], [0], [0], [0]])

result = krylov_method(A, v)
print(result)
print(np.poly1d(np.insert(result,0,1).flatten()))
#### answer:
'''
[[-20.]
 [-56.]
 [-40.]
 [ -4.]]
   4      3      2
1 x - 20 x - 56 x - 40 x - 4

'''

[[-20.]
 [-56.]
 [-40.]
 [ -4.]]
   4      3      2
1 x - 20 x - 56 x - 40 x - 4


'\n[[-20.]\n [-56.]\n [-40.]\n [ -4.]]\n   4      3      2\n1 x - 20 x - 56 x - 40 x - 4\n\n'